In [1]:
list_authors = [2150992,2370123,63349824,50018709,12149340,48203381,14554912,26919255,22267216,15446425,48114135,2243973]
id_co_auteurs = [29091850,7137580,20979755,20988767,29836419,63349824,50018709,63421194,63939275,12149340,48203381,14554912,26919255,22267216,15446425,3860521,4131094,30640054,21937449,3700900,8260331,21146974,24013573,30334301,33473390,33428872,3265659,3334924,34324840,38508667,47898726,2093484,4154681,6582591,40252978,38188043,40640913,47943401,57499050,63042094,2243973,2094821,2294353,5734840,9618818,18634145,19182991]

In [3]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException ,TimeoutException,NoSuchWindowException
from bs4 import BeautifulSoup
import time
import pandas as pd
import csv
import re

### Si vous n'utilisez pas le réseau wifi de l'ENSA, alors il est obligatoire de vous vous authentifier

In [5]:
# Spécifiez le chemin vers EdgeDriver
edgedriver_path = r"edgedriver_win64\msedgedriver.exe"

# Configurez les options pour Edge
options = Options()
# options.add_argument("--start-maximized")  # Lancer Edge en mode maximisé
# options.add_experimental_option("detach", True)  # Garde le navigateur ouvert après exécution

# Initialisez le service EdgeDriver
service = Service(executable_path=edgedriver_path)

# Lancez le navigateur Edge
driver = webdriver.Edge(service=service)

# Ouvrez l'URL cible
driver.get("https://www.webofscience.com.eressources.imist.ma/")
# driver.get("https://www.webofscience.com/wos/author/record/") # Au cas de l'utilisation du réseau Wifi de l'ENSA

In [7]:
# Trouver les champs d'email et de mot de passe sur la page, y saisir les informations d'identification, puis soumettre le formulaire.
email_field = driver.find_element(By.ID, "email")
password_field = driver.find_element(By.ID, "password")

email_field.send_keys('meryem.bouali@usms.ac.ma') # Saisir l'adresse email
password_field.send_keys('M135147898') # Saisir le mot de passe

password_field.send_keys(Keys.RETURN)

In [14]:
wait = WebDriverWait(driver, 10)

def get_author_information(id):
    driver.get(f"https://www.webofscience.com.eressources.imist.ma/wos/author/record/{id}")
    # driver.get(f"https://www.webofscience.com/wos/author/record/{id}")
    time.sleep(5)
    scroll_slowly(driver)
    
    try:
        infos = {
            'ID de l\'Auteur' : id,
            'nom_complet' : None,
            'pays_affiliation' : None,
            'co_auteurs' : 0,
            'H-Index' : 0,
            'Sum of Times Cited' : 0
        }
        
        # Récupération des co-auteurs
        try:
            co_auteurs = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'authors-list-link')))
            infos['co_auteurs'] = [auteur.text for auteur in co_auteurs]
            id_co_auteur = [auteur.get_attribute('href').split('/')[-1] for auteur in co_auteurs]                        
        except:
            infos['co_auteurs'] = []
            id_co_auteur = []
            print('Les co-auteurs n\'existent pas')
        
        # Récupération des métriques
        try:
            metric_descriptor = WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'wat-author-metric-descriptor')))
            for metric in metric_descriptor:
                if metric.text in ['H-Index', 'Sum of Times Cited']:
                    value = metric.find_element(By.XPATH, './preceding-sibling::div').text
                    infos[metric.text] = value
        except:
            print('H-index ou bien citation n\'existent pas')
        
        # Récupération du pays d'affiliation
        try:
            value = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'more-details'))).text.split(',')[-1].strip()
            infos['pays_affiliation'] = value
        except:
            print('Pays d\'affiliation n\'existe pas')
        
        # Récupération du nom complet
        try:
            value = WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'wat-author-name'))).text
            infos['nom_complet'] = value
        except:
            print('Le nom n\'existe pas')
            
        return infos , id_co_auteur
    except:
        print('Erreur lors de l\'attente du chargement de la page')

def extract_article_details(driver, article_link):
    driver.get(article_link)
    time.sleep(4)  # Pause pour laisser le temps à la page de se charger
    scroll_slowly(driver)  # Faire défiler la page lentement pour charger le contenu
    infos = {}  # Dictionnaire pour stocker les informations de l'article
    
    try:
        # Tente de trouver l'élément
        element = driver.find_element(By.CLASS_NAME, 'error-content')
        return 'break'
    except NoSuchElementException:
        print('')
    
    # Titre de l'article
    try:
        infos['Titre de l’article'] = wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, 'title'))
        ).text
    except Exception as e:
        infos['Titre de l’article'] = None
        print("Erreur pour Titre de l’article:")
    
    # Auteur
    try:
        # Récupérer les noms des auteurs ou les mots-clés
        authors = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//a[starts-with(@id,'SumAuthTa-DisplayName-author-en-')]")))
        infos['Auteurs'] = ' ; '.join([element.text for element in authors])  # Joindre les noms
    except Exception as e:
        infos['Auteur'] = None
        print("Erreur pour Auteur:" )
    
    # Date de publication 
    class_names = ['FullRTa-pubdate', 'FullRTa-earlyAccess']
    for class_name in class_names:
        try:
            infos['Date de publication'] = wait.until(EC.presence_of_element_located((By.ID, class_name))).text
            break  # Sortir de la boucle si trouvé
        except Exception as e:
            infos['Date de publication'] = None
            print("Erreur pour date:" )
            
    # Source
    class_names = ['summary-source-title-link', 'summary-source-title']
    for class_name in class_names:
        try:
            infos['Titre de la source'] = wait.until(EC.presence_of_element_located((By.CLASS_NAME, class_name))).text
            break  # Sortir de la boucle si trouvé
        except Exception as e:
            infos['Titre de la source'] = None
            print("Erreur pour Source:" )
    
    # Mots-clés
    mots_cles = ''
    id_names = ['FRkeywordsTa-keyWordsPlusLink-','FRkeywordsTa-authorKeywordLink-']
    for id_name in id_names:
        try: 
            value = wait.until(EC.presence_of_all_elements_located((By.XPATH, f"//a[starts-with(@id,'{id_name}')]")))
            mots_cles = mots_cles +' ' + ' ; '.join([element.text for element in value])  # Joindre les noms
        except Exception as e:
            infos['Mots-clés'] = None
            print("Erreur pour Mots-clés:" )

    infos['Mots-clés'] = mots_cles
    
    # Nombre de citations
    try:
        # Attendre et récupérer le nombre de citations
        infoElem = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'citation-count'))).text.split('\n')
        if len(infoElem) > 1 and infoElem[1] == 'Cited References':
            infos['Nombre de citations'] = 0  # Pas de citations
        else:
            infos['Nombre de citations'] = infoElem[0]  # Nombre de citations
    except Exception as e:
        print(f"Erreur lors de la récupération de 'citation-count': {e}")
        infos['Nombre de citations'] = None  # Valeur par défaut en cas d'erreur

    # DOI
    try:
        infos['DOI'] = wait.until(
            EC.presence_of_element_located((By.ID, 'FullRTa-DOI'))
        ).text
    except Exception as e:
        infos['DOI'] = None
        print("Erreur pour DOI:" )
    
    # Résumé
    try:
        infos['Résumé'] = wait.until(
            EC.presence_of_element_located((By.ID, 'FullRTa-abstract-basic'))
        ).text
    except Exception as e:
        infos['Résumé'] = None
        print("Erreur pour Résumé:" )
    
    # Type de document
    try:
        infos['Type de document'] = wait.until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="FullRTa-doctype-0"]'))
        ).text
    except Exception as e:
        infos['Type de document'] = None
        print("Erreur pour Type de document:" )
        
    try:
        # Récupérer l'ISSN de l'article
        infos['issn'] = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'value.section-label-data.text-color'))).text
    except (TimeoutException, NoSuchElementException):
        print("L'élément ISSN n'a pas été trouvé.")  
        
    return infos  


# Fonction récupère les liens des articles en parcourant les pages d'une liste d'articles. 
def get_article_titles():
    titles = []
    while True:
        articles = driver.find_elements(By.CLASS_NAME, 'title')
        for article in articles:
            titles.append(article.get_attribute('href'))
        try:
            next_button = driver.find_element(By.XPATH , '//button[@data-ta="next-page-button"]')
            if 'mat-button-disabled' in next_button.get_attribute('class'):
                break  
            else:
                next_button.click()
                scroll_slowly(driver, scroll_pause_time=0.2, scroll_increment=100)
        except NoSuchElementException:
            break  
    return titles
    
# Fonction qui fait défiler lentement la page dans le navigateur.
def scroll_slowly(driver, scroll_pause_time=0.1, scroll_increment=90):
    
    # Récupère la hauteur totale de la page.
    last_height = driver.execute_script("return document.body.scrollHeight")
    current_scroll_position = 0  # Commence en haut de la page
    
    while current_scroll_position < last_height:
        # Fait défiler la page par petits incréments (scroll_increment)
        current_scroll_position += scroll_increment
        driver.execute_script(f"window.scrollTo(0, {current_scroll_position});")
        
        # Attend un court instant entre les défilements
        time.sleep(scroll_pause_time)
        
        # Met à jour la hauteur totale au cas où la page chargerait dynamiquement plus de contenu
        last_height = driver.execute_script("return document.body.scrollHeight")


In [11]:
def Search_journal_info(driver, issn):
    
    # Accéder à la page avec le formulaire de recherche
    driver.get("https://www.scimagojr.com/")

    # Attendre que l'input de recherche soit présent
    try:               
        search_input = wait.until(
            EC.presence_of_element_located((By.ID, "searchinput"))
        )
        # Entrer l'ISSN et simuler un appui sur la touche "Entrée"
        search_input.clear()
        search_input.send_keys(issn, Keys.RETURN)

        # Attendre que les résultats s'affichent et cliquer
        wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "search_results"))
        ).click()

    except Exception as e:
        print(f"An error occurred: {e}")
        
def extract_quartile(soup):
    # Extraction de la métrique Quartile
    quartile_data = {}
    dash = soup.findAll('div', class_='dashboard')
    if dash:
        quart_dash = dash[0].findAll('div', class_="cellslide")
        if quart_dash:
            last_quartile_row = quart_dash[1].find('tbody').find_all('tr')[-1].find_all('td')
            if len(last_quartile_row) == 3:
                quartile_data = {
#                     "category": last_quartile_row[0].text,
                    "year": last_quartile_row[1].text,
                    "quartile_value": last_quartile_row[2].text
                }
    return quartile_data

def extract_sjr(soup):
    # Extraction de la métrique SJR
    sjr_data = {}
    dash = soup.findAll('div', class_='dashboard')
    if len(dash) > 1:
        sjr_dash = dash[1].findAll('div', class_="cellslide")[1]
        sjr_row = sjr_dash.find('tbody').find_all('tr')[-1].find_all('td')
        if len(sjr_row) == 2:
            sjr_data = {
                "year": sjr_row[0].text,
                "sjr_value": sjr_row[1].text
            }
    return sjr_data

def extract_impact_factor(soup):
    # Extraction de la métrique Facteur d'Impact
    impact_data = {}
    dash = soup.findAll('div', class_='dashboard')
    if len(dash) > 1:
        impact_dash = dash[1].findAll('div', class_="cellslide")[5]
        impact_row = impact_dash.find('tbody').find_all('tr')[-1].find_all('td')
        if len(impact_row) == 3:
            impact_data = {
                "year": impact_row[1].text,
                "impact_factor_value": impact_row[2].text
            }
    return impact_data

def extract_journal_metrics(driver, issn_list):
    journals_info = []    
    
    for issn in issn_list:
        try:
            # Utiliser Search_journal_info pour obtenir les metriques
            Search_journal_info(driver, issn)

            # Scraper le h-index
            try:
                hindexnumber = wait.until(
                    EC.presence_of_element_located((By.CLASS_NAME, "hindexnumber"))
                ).text
            except TimeoutException:
                hindexnumber = "N/A"

            # Scraper le scope en retirant la dernière ligne non souhaitée
            try:
                scope_pt = wait.until(
                    EC.presence_of_element_located((By.CLASS_NAME, "fullwidth"))
                ).text.split("\n")[1]
            except (TimeoutException, IndexError):
                scope_pt = "N/A"

            # Obtenir le code HTML de la page et l'analyser avec BeautifulSoup
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')

            # Extraire les autres informations de la revue
            journal_data = {
                "issn": issn,
                "scope": scope_pt,
                "index": "WOS",
                "h_index": hindexnumber,
                "quartile": extract_quartile(soup) or "N/A",
                "sjr": extract_sjr(soup) or {"sjr_value": "N/A", "year": "N/A"},
                "impact_factor": extract_impact_factor(soup) or {"impact_factor_value": "N/A", "year": "N/A"}
            }

            # Ajouter les données de la revue à la liste des informations de revues
            journals_info.append(journal_data)
            print("Get Journal info for issn ====> ", issn)

        except Exception as e:
            print(f"An error occurred while processing ISSN {issn}: {e}")
            journals_info.append({
                "issn": issn,
                "scope": "N/A",
                "index": "Scopus",
                "h_index": "N/A",
                "quartile": "N/A",
                "sjr": {"sjr_value": "N/A", "year": "N/A"},
                "impact_factor": {"impact_factor_value": "N/A", "year": "N/A"}
            })
            
    return list(journals_info)

In [ ]:
all_data = {} 
try:
    for i in range(len(list_authors)):
        # Récupérer les informations de l'auteur
        _, ids = get_author_information(list_authors[i])
        ids.append(int(list_authors[i]))
        
        for id in ids:
            if int(id) not in id_co_auteurs:
                print('type de id not exist ', type(id))
                print('id not exist ', id)
                author_data,_ = get_author_information(id)
                id_co_auteurs.append(int(id))
                
                while True:
                    # Obtenir les titres des articles de l'auteur
                    articles = get_article_titles()
                    if len(articles) > 0:
                        break  # Sortir de la boucle si des articles sont trouvés
                    else:
                        driver.refresh()
                        time.sleep(1)
                        scroll_slowly(driver) 
                    
                all_articles_data = []
                issns = []
                if articles != 0 : 
                    for article_link in articles:
                        flag = 0  # Drapeau pour vérifier si l'ISSN a été trouvé
                        while True:
                            # Extraire les détails de l'article
                            data_article = extract_article_details(driver, article_link)
                            
                            if data_article == 'break':
                                break
                                
                            if 'issn' in data_article.keys():
                                all_articles_data.append(data_article)  # Ajouter les données de l'article à la liste
                                issns.append(data_article['issn'])
                                break  # Sortir de la boucle si l'ISSN est trouvé
                            else:   
                                flag += 1  # Incrémenter le drapeau
                                scroll_slowly(driver)  # Faire défiler la page
                                if flag == 1:
                                    data_article['issn'] = None  # Assigner None si l'ISSN n'est pas trouvé
                                    all_articles_data.append(data_article)  # Ajouter les données de l'article même sans ISSN
                                    issns.append(data_article['issn'])
                                    break
                            
                    # Extraire les données des journaux
                    journals_data = extract_journal_metrics(driver, issns)
                    for j in range(len(all_articles_data)):
                        # Lier les données des articles avec les données des journaux par ISSN
                        if all_articles_data[j]['issn'] == journals_data[j]['issn']:
                            all_articles_data[j]['journal_data'] = journals_data[j]  # Ajouter les données du journal
                            del all_articles_data[j]['issn']  # Supprimer l'ISSN des données de l'article
                           
                    author_data['Articles'] = all_articles_data  # Ajouter les données des articles à l'auteur
                all_data[id] = author_data  # Ajouter les données de l'auteur au dictionnaire final 
except NoSuchWindowException:
        print("Browser window was closed. Restarting the session.")

type de id not exist  <class 'str'>
id not exist  7440450
Les co-auteurs n'existent pas


Erreur pour DOI:


Erreur pour Mots-clés:

Erreur pour Mots-clés:
Erreur pour Mots-clés:



Erreur pour Source:
Erreur pour DOI:

Erreur pour Source:
Erreur pour DOI:

Erreur pour Mots-clés:

Erreur pour DOI:

Erreur pour Mots-clés:
Erreur pour DOI:


Erreur pour Source:
Erreur pour Mots-clés:
Erreur pour Mots-clés:
Erreur pour DOI:
Erreur pour Résumé:
L'élément ISSN n'a pas été trouvé.

Erreur pour Source:
Erreur pour Mots-clés:
Erreur pour Mots-clés:
Erreur pour DOI:
Erreur pour Résumé:
L'élément ISSN n'a pas été trouvé.

Erreur pour Mots-clés:
Erreur pour Mots-clés:

Erreur pour Mots-clés:
Erreur pour Mots-clés:

Erreur pour Mots-clés:
Erreur pour Mots-clés:

Erreur pour Mots-clés:
Erreur pour Mots-clés:
Erreur pour DOI:

Erreur pour Mots-clés:
Erreur pour Mots-clés:

Erreur pour Mots-clés:
Erreur pour Mots-clés:
Erreur pour DOI:
Erreur pour Résumé:

Erreur pour Mots-clés:
Erreur pour Mots-clés

In [51]:
import json

# Enregistrer les données dans un fichier JSON
with open('authors_data_wos.json', 'a', encoding='utf-8') as json_file:
    json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print("Données enregistrées dans author_data.json")

Données enregistrées dans author_data.json


In [209]:
import pandas as pd

with open('authors_data_wos.json' , encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)
df = df.transpose()
df

In [211]:
df['Journal'] = [[] for _ in range(df.shape[0])]

In [215]:
for index in df.index:
    journal_data = []
    try:
        for j in range(len(df['Articles'][index])):
            journal_data.append(df['Articles'][index][j].pop('journal_data'))
            
        df.at[index, 'Journal'] = journal_data  # Utilisez .at pour un accès plus direct
    except:
        print("Journal data n'existe pas")

Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data n'existe pas
Journal data

In [217]:
journal_data

[]

In [219]:
# df['Articles'][29091850][0].pop('journal_data')
df

,ID de l'Auteur,nom_complet,pays_affiliation,co_auteurs,H-Index,Sum of Times Cited,Articles,Journal
29091850,29091850,"Aghriche, Ahmed",MOROCCO,[],2,26,[{'Titre de l’article': 'OSCILLATIONS INDUCED ...,"[{'issn': '1937-1632', 'scope': 'Series S of D..."
7137580,7137580,"Dargham, Abdelmajid",MOROCCO,[],1,2,[{'Titre de l’article': 'REDUCING THE INTERPRO...,"[{'issn': '1895-1767', 'scope': 'The area of s..."
20979755,20979755,"El Bannay, Omar",MOROCCO,[],2,27,[{'Titre de l’article': 'Determining the potat...,"[{'issn': '0957-0233', 'scope': 'Measurement ..."
20988767,20988767,"Fatmi, Nadia Idrissi",MOROCCO,[],1,4,[{'Titre de l’article': 'MATHEMATICAL MODEL AN...,"[{'issn': '2052-2541', 'scope': 'Information n..."
29836419,29836419,"Ifzarne, Aziz",MOROCCO,[],3,32,[{'Titre de l’article': 'Quarter-Tone Music: A...,"[{'issn': '0302-9743', 'scope': 'This distingu..."
63349824,63349824,"Lamghari, Nidal",MOROCCO,[],0,0,[{'Titre de l’article': 'Subword recognition i...,"[{'issn': '1433-2833', 'scope': 'The large num..."
50018709,50018709,noureddine aboutabit,None,"[Lamghari, Nidal, Dahbali, Mohamed, Mait, Hind...",1,1,[{'Titre de l’article': 'Subword recognition i...,"[{'issn': '1433-2833', 'scope': 'The large num..."
63421194,63421194,"Dahbali, Mohamed",MOROCCO,[],0,0,[{'Titre de l’article': 'Subword recognition i...,"[{'issn': '1433-2833', 'scope': 'The large num..."
63939275,63939275,"Mait, Hind Ait",MOROCCO,[],0,0,[{'Titre de l’article': 'Unsupervised VAD meth...,"[{'issn': '0952-8091', 'scope': 'IJCAT address..."
12149340,12149340,"Nasri, Mohamed",MOROCCO,[],2,11,[{'Titre de l’article': 'Semantic Analysis of ...,"[{'issn': None, 'scope': 'N/A', 'index': 'WOS'..."


In [221]:
df.to_csv('authors_data_wos.csv',encoding='utf-8',index=False)